# Docking molecular entre epítopos y los MHC de clase I y clase II
El docking molecular de los complejos epítopo-MHC (eMHC) se llevó a cabo por dos servidores en línea para mejorar la precisión. 

In [1]:
# Instalar el paquete 'selenium'
!pip install selenium

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 1.9 MB/s eta 0:00:00m eta 0:00:010:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 1.0 MB/s eta 0:00:000:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 895.4 kB/s eta 0:00:001m913.9 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.0 MB/s eta 0:00:00 MB/s eta 0:00:01


In [1]:
# Importar los módulos necesarios de 'selenium'
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import time

Primero, se utilizó el servidor **ClusPro v2.0** (https://cluspro.bu.edu/home.php) (*Kozakov et al., 2017*), del cual se seleccionó el <u>modelo de coeficientes balanceados</u>, y se eligió el complejo con menor puntuación ponderada por cada *docking* enviado al servidor. 

In [2]:
# Definir la función para la simulación del acoplamiento proteína-ligando en ClusPro v2.0
def cluspro_docking(HLA, ligando, output):
    # Establecer la ubicación del WebDriver
    s = Service('/usr/local/bin/chromedriver')
    # Inicializar el Selenium WebDriver
    driver = webdriver.Chrome(service = s)
    # Visitar al servidor ClusPro v2.0
    driver.get('https://cluspro.bu.edu/home.php')
    time.sleep(5)
    # Editar los parámetros
    ## Nombrar el trabajo
    job_name = driver.find_elements(by = By.NAME, value = 'jobname')
    
    ## Cargar los archivos input
    show_receptor = driver.find_element(by = By.ID, value = 'showrecfile')
    import_receptor = driver.find_elements(by = By.ID, value = 'rec')
    
    show_ligand = driver.find_element(by = By.ID, value = 'showligfile')
    import_ligand = driver.find_element(by = By.ID, value = 'lig')

    ## Definir el archivo de salida donde estarán los links de los resultados
    resultados_ClusPro = f'/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_Cluspro/{output}'

    # Realizar el docking molecular
    for HLA_IorII in HLA:
        for name in job_name:
            name.send_keys(f'{HLA_IorII}//{ligando}')
        for receptor in import_receptor:
            show_receptor.click()
            time.sleep(1)
            receptor.send_keys(f'/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/HLA/{HLA_IorII}.pdb')
            
            show_ligand.click()
            time.sleep(1)
            import_ligand.send_keys(f'/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/Epítopos/{ligando}.pdb')
            # Entregar la data
            driver.find_element(by = By.NAME, value = 'action').click()
            time.sleep(2)
            ## Obtener los resultados de la predicción
            get_url = driver.current_url
            result_output = driver.find_element(By.XPATH, value = '/html/body/div[1]/div[2]/table/tbody/tr[2]').text
            with open(resultados_ClusPro, 'a') as file:
                print(f'{result_output}', file = file)
                print('................', file = file)
            driver.get('https://cluspro.bu.edu/home.php')
            time.sleep(2)               
            job_name = driver.find_elements(by = By.NAME, value = 'jobname')
            show_receptor = driver.find_element(by = By.ID, value = 'showrecfile')
            import_receptor = driver.find_elements(by = By.ID, value = 'rec')
            show_ligand = driver.find_element(by = By.ID, value = 'showligfile')
            import_ligand = driver.find_element(by = By.ID, value = 'lig')
            resultados_ClusPro = f'/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_Cluspro/{output}'
    driver.close()
    
# Visualizar los IDs generados
def result_ClusPro(resultado_ClusPro):
    with open(resultado_ClusPro) as f:
        docking_cluspro_resultados = f.read()
        print(docking_cluspro_resultados)

In [3]:
# Definir los nombres de los archivos input donde están las estructuras 3D de los HLA clase I y clase II
HLA_I = ['HLA-A*01:01_(6MPP)', 'HLA-A*02:01_(4U6X)', 'HLA-A*03:01_(3RL1)', 'HLA-A*24:02_(7JYV)', 'HLA-A*31:01',
        'HLA-B*35:01_(4PRN)', 'HLA-B*44:03_(3DX7)', 'HLA-B*51:01_(1E28)', 'HLA-C*03:04', 'HLA-C*04:01',
        'HLA-C*07:01', 'HLA-C*07:02_(5VGE)']
HLA_II = ['HLA-DRB1*03:01', 'HLA-DRB1*04:07', 'HLA-DRB1*07:01_(7Z0Q)', 'HLA-DRB1*08:02', 'HLA-DRB1*14:02_(6ATF)']

In [13]:
# Realizar el docking entre los alelos HLA I y el epítopo CTL_NS3 (TVWFVPSIK)
cluspro_docking(HLA_I, 'CTL_NS3_TVWFVPSIK', 'CTL_NS3')
# Realizar el docking entre los alelos HLA II y el epítopo CTL_NS3 (TVWFVPSIK)
cluspro_docking(HLA_II, 'CTL_NS3_TVWFVPSIK', 'CTL_NS3')
# Visualizar los IDs generados
result_ClusPro('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_Cluspro/CTL_NS3')

970504 HLA-A*01:01_(6MPP)// elcornevic processing pdb files
................
970505 HLA-A*02:01_(4U6X)// elcornevic processing pdb files
................
970506 HLA-A*03:01_(3RL1)// elcornevic processing pdb files
................
970507 HLA-A*24:02_(7JYV)// elcornevic processing pdb files
................
970508 HLA-A*31:01//CTL_NS3 elcornevic processing pdb files
................
970509 HLA-B*35:01_(4PRN)// elcornevic processing pdb files
................
970510 HLA-B*44:03_(3DX7)// elcornevic processing pdb files
................
970511 HLA-B*51:01_(1E28)// elcornevic processing pdb files
................
970512 HLA-C*03:04//CTL_NS3 elcornevic processing pdb files
................
970513 HLA-C*04:01//CTL_NS3 elcornevic processing pdb files
................
970514 HLA-C*07:01//CTL_NS3 elcornevic processing pdb files
................
970515 HLA-C*07:02_(5VGE)// elcornevic processing pdb files
................
971043 HLA-DRB1*03:01//CTL_ elcornevic processing pdb files
................

In [17]:
# Realizar el docking entre los alelos HLA I y el epítopo CTL_NS5 (MYFHRRDLR)
cluspro_docking(HLA_I, 'CTL_NS5_MYFHRRDLR', 'CTL_NS5')
# Realizar el docking entre los alelos HLA II y el epítopo CTL_NS5 (MYFHRRDLR)
cluspro_docking(HLA_II, 'CTL_NS5_MYFHRRDLR', 'CTL_NS5')
# Visualizar los IDs generados
result_ClusPro('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_Cluspro/CTL_NS5')

971107 HLA-A*01:01_(6MPP)// elcornevic processing pdb files
................
971108 HLA-A*02:01_(4U6X)// elcornevic processing pdb files
................
971109 HLA-A*03:01_(3RL1)// elcornevic processing pdb files
................
971110 HLA-A*24:02_(7JYV)// elcornevic processing pdb files
................
971111 HLA-A*31:01//CTL_NS5 elcornevic processing pdb files
................
971112 HLA-B*35:01_(4PRN)// elcornevic processing pdb files
................
971113 HLA-B*44:03_(3DX7)// elcornevic processing pdb files
................
971114 HLA-B*51:01_(1E28)// elcornevic processing pdb files
................
971115 HLA-C*03:04//CTL_NS5 elcornevic processing pdb files
................
971116 HLA-C*04:01//CTL_NS5 elcornevic processing pdb files
................
971117 HLA-C*07:01//CTL_NS5 elcornevic processing pdb files
................
971118 HLA-C*07:02_(5VGE)// elcornevic processing pdb files
................
971119 HLA-DRB1*03:01//CTL_ elcornevic processing pdb files
................

In [15]:
# Realizar el docking entre los alelos HLA I y el epítopo HTL_NS1 (RAVHADMGYWIE)
cluspro_docking(HLA_I, 'HTL_NS1_RAVHADMGYWIE', 'HTL_NS1')
# Realizar el docking entre los alelos HLA II y el epítopo HTL_NS1 (RAVHADMGYWIE)
cluspro_docking(HLA_II, 'HTL_NS1_RAVHADMGYWIE', 'HTL_NS1')
# Visualizar los IDs generados
result_ClusPro('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_Cluspro/HTL_NS1')

1009659 HLA-A*01:01_(6MPP)// elcornevic processing pdb files
................
1009660 HLA-A*02:01_(4U6X)// elcornevic processing pdb files
................
1009661 HLA-A*03:01_(3RL1)// elcornevic processing pdb files
................
1009662 HLA-A*24:02_(7JYV)// elcornevic processing pdb files
................
1009663 HLA-A*31:01//HTL_NS1 elcornevic processing pdb files
................
1009664 HLA-B*35:01_(4PRN)// elcornevic processing pdb files
................
1009665 HLA-B*44:03_(3DX7)// elcornevic processing pdb files
................
1009666 HLA-B*51:01_(1E28)// elcornevic processing pdb files
................
1009667 HLA-C*03:04//HTL_NS1 elcornevic processing pdb files
................
1009668 HLA-C*04:01//HTL_NS1 elcornevic processing pdb files
................
1009669 HLA-C*07:01//HTL_NS1 elcornevic processing pdb files
................
1009670 HLA-C*07:02_(5VGE)// elcornevic processing pdb files
................
1009672 HLA-DRB1*03:01//HTL_ elcornevic processing pdb files
...

In [23]:
# Realizar el docking entre los alelos HLA I y el epítopo HTL_NS1 (WDFVVTTDISEM)
cluspro_docking(HLA_I, 'HTL_NS3_1_WDFVVTTDISEM', 'HTL_NS3_1')
# Realizar el docking entre los alelos HLA II y el epítopo HTL_NS3_1 (WDFVVTTDISEM)
cluspro_docking(HLA_II, 'HTL_NS3_1_WDFVVTTDISEM', 'HTL_NS3_1')
# Visualizar los IDs generados
result_ClusPro('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_Cluspro/HTL_NS3_1')

971192 HLA-A*01:01_(6MPP)// elcornevic processing pdb files
................
971194 HLA-A*02:01_(4U6X)// elcornevic processing pdb files
................
971195 HLA-A*03:01_(3RL1)// elcornevic processing pdb files
................
971196 HLA-A*24:02_(7JYV)// elcornevic processing pdb files
................
971198 HLA-A*31:01//HTL_NS3 elcornevic processing pdb files
................
971199 HLA-B*35:01_(4PRN)// elcornevic processing pdb files
................
971200 HLA-B*44:03_(3DX7)// elcornevic processing pdb files
................
971202 HLA-B*51:01_(1E28)// elcornevic processing pdb files
................
971203 HLA-C*03:04//HTL_NS3 elcornevic processing pdb files
................
971204 HLA-C*04:01//HTL_NS3 elcornevic processing pdb files
................
971206 HLA-C*07:01//HTL_NS3 elcornevic processing pdb files
................
971207 HLA-C*07:02_(5VGE)// elcornevic processing pdb files
................
971299 HLA-DRB1*03:01//HTL_ elcornevic processing pdb files
................

In [27]:
# Realizar el docking entre los alelos HLA I y el epítopo HTL_NS3_2 (DWDFVVTTDISE)
cluspro_docking(HLA_I, 'HTL_NS3_2_DWDFVVTTDISE', 'HTL_NS3_2')
# Realizar el docking entre los alelos HLA II y el epítopo HTL_NS3_2 (DWDFVVTTDISE)
cluspro_docking(HLA_II, 'HTL_NS3_2_DWDFVVTTDISE', 'HTL_NS3_2')
# Visualizar los IDs generados
result_ClusPro('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_Cluspro/HTL_NS3_2')

971337 HLA-A*01:01_(6MPP)// elcornevic processing pdb files
................
971338 HLA-A*02:01_(4U6X)// elcornevic processing pdb files
................
971339 HLA-A*03:01_(3RL1)// elcornevic processing pdb files
................
971340 HLA-A*24:02_(7JYV)// elcornevic processing pdb files
................
971341 HLA-A*31:01//HTL_NS3 elcornevic processing pdb files
................
971342 HLA-B*35:01_(4PRN)// elcornevic processing pdb files
................
971343 HLA-B*44:03_(3DX7)// elcornevic processing pdb files
................
971344 HLA-B*51:01_(1E28)// elcornevic processing pdb files
................
971345 HLA-C*03:04//HTL_NS3 elcornevic processing pdb files
................
971346 HLA-C*04:01//HTL_NS3 elcornevic processing pdb files
................
971347 HLA-C*07:01//HTL_NS3 elcornevic processing pdb files
................
971348 HLA-C*07:02_(5VGE)// elcornevic processing pdb files
................
971362 HLA-DRB1*03:01//HTL_ elcornevic processing pdb files
................

Posteriormente, se empleó el servidor **HawkDock** (http://cadd.zju.edu.cn/hawkdock/) (*Weng et al., 2019*), donde las puntuaciones y energía mas bajas corresponden a las mejores puntuaciones. 

In [10]:
# Definir la función para la simulación del acoplamiento proteína-ligando en HawkDock
def hawkdock_docking(HLA, ligando, output):
    # Establecer la ubicación del WebDriver
    s = Service('/usr/local/bin/chromedriver')
    # Inicializar el Selenium WebDriver
    driver = webdriver.Chrome(service = s)
    # Visitar al servidor HawkDock
    driver.get('http://cadd.zju.edu.cn/hawkdock/')
    time.sleep(2)
    # Editar los parámetros
    ## Nombrar el trabajo
    job_name = driver.find_elements(by = By.ID, value = 'jobname_HawkDock')
    
    ## Insertar el correo
    email = driver.find_element(by = By.NAME, value = 'email')
    
    ## Cargar los archivos input
    import_receptor = driver.find_elements(by = By.ID, value = 'rec_PDBfile1')    
    import_ligand = driver.find_element(by = By.ID, value = 'lig_PDBfile1')

    ## Definir el archivo de salida donde estarán los links de los resultados
    resultados_HawkDock = f'/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_HawkDock/{output}'

    # Realizar el docking molecular
    for HLA_IorII in HLA:
        for name in job_name:
            name.send_keys(f'{HLA_IorII}//{ligando}')
            email.send_keys('victor.cornejo@unmsm.edu.pe')
        for receptor in import_receptor:
            receptor.send_keys(f'/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/HLA/{HLA_IorII}.pdb')
            import_ligand.send_keys(f'/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/Epítopos/{ligando}.pdb')
            # Entregar la data
            driver.find_element(by = By.ID, value = 'submit_HawkDock_PDBcalc').click()
            time.sleep(15)
            ## Obtener los resultados de la predicción
            get_url = driver.current_url
            driver.find_element(by = By.XPATH, value = '/html/body/nav/div/div[2]/ul/li[3]/a').click()
            time.sleep(2)
            get_url = driver.current_url
            result_output = driver.find_element(By.XPATH, value = '/html/body/div[1]/div/div/div/div[2]/table/tbody/tr[1]').text
            with open(resultados_HawkDock, 'a') as file:
                print(f'{result_output}', file = file)
                print('................', file = file)
            driver.get('http://cadd.zju.edu.cn/hawkdock/')
            time.sleep(2)
            job_name = driver.find_elements(by = By.ID, value = 'jobname_HawkDock')
            email = driver.find_element(by = By.NAME, value = 'email')
            import_receptor = driver.find_elements(by = By.ID, value = 'rec_PDBfile1')
            import_ligand = driver.find_element(by = By.ID, value = 'lig_PDBfile1')
            resultados_HawkDock = f'/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_HawkDock/{output}'
    driver.close()
    
# Visualizar los IDs generados
def result_HawkDock(resultado_HawkDock):
    with open(resultado_HawkDock) as f:
        docking_hawkdock_resultados = f.read()
        print(docking_hawkdock_resultados)

In [4]:
# Realizar el docking entre los alelos HLA I y el epítopo CTL_NS3 (TVWFVPSIK)
hawkdock_docking(HLA_I, 'CTL_NS3_TVWFVPSIK', 'CTL_NS3')
# Realizar el docking entre los alelos HLA II y el epítopo CTL_NS3 (TVWFVPSIK)
hawkdock_docking(HLA_II, 'CTL_NS3_TVWFVPSIK', 'CTL_NS3')
# Visualizar los IDs generados
result_HawkDock('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_HawkDock/CTL_NS3')

147145 HLAA0101_6MPPCTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:15:53 (UTC + 8 hours)
................
147146 HLAA0201_4U6XCTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:16:17 (UTC + 8 hours)
................
147147 HLAA0301_3RL1CTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:16:40 (UTC + 8 hours)
................
147148 HLAA2402_7JYVCTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:17:02 (UTC + 8 hours)
................
147149 HLAA3101CTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:17:26 (UTC + 8 hours)
................
147150 HLAB3501_4PRNCTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:17:48 (UTC + 8 hours)
................
147151 HLAB4403_3DX7CTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:18:11 (UTC + 8 hours)
................
147152 HLAB5101_1E28CTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:18:33 (UTC + 8 hours)
................
147153 HLAC0304CTL_NS3_TVWFVPSIK HawkDock Waiting 2023-07-09 21:18:55 (UTC + 8 hours)
................
147154 HLAC0401CTL_NS3_TVWFVPSIK HawkD

In [5]:
# Realizar el docking entre los alelos HLA I y el epítopo CTL_NS5 (MYFHRRDLR)
hawkdock_docking(HLA_I, 'CTL_NS5_MYFHRRDLR', 'CTL_NS5')
# Realizar el docking entre los alelos HLA II y el epítopo CTL_NS5 (MYFHRRDLR)
hawkdock_docking(HLA_II, 'CTL_NS5_MYFHRRDLR', 'CTL_NS5')
# Visualizar los IDs generados
result_HawkDock('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_HawkDock/CTL_NS5')

147225 HLAA0101_6MPPCTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:34:48 (UTC + 8 hours)
................
147226 HLAA0201_4U6XCTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:35:10 (UTC + 8 hours)
................
147227 HLAA0301_3RL1CTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:35:31 (UTC + 8 hours)
................
147228 HLAA2402_7JYVCTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:35:53 (UTC + 8 hours)
................
147229 HLAA3101CTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:36:16 (UTC + 8 hours)
................
147230 HLAB3501_4PRNCTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:36:37 (UTC + 8 hours)
................
147231 HLAB4403_3DX7CTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:36:59 (UTC + 8 hours)
................
147232 HLAB5101_1E28CTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:37:23 (UTC + 8 hours)
................
147233 HLAC0304CTL_NS5_MYFHRRDLR HawkDock Waiting 2023-07-10 01:37:45 (UTC + 8 hours)
................
147234 HLAC0401CTL_NS5_MYFHRRDLR HawkD

In [14]:
# Realizar el docking entre los alelos HLA I y el epítopo HTL_NS1 (RAVHADMGYWIE)
hawkdock_docking(HLA_I, 'HTL_NS1_RAVHADMGYWIE', 'HTL_NS1')
# Realizar el docking entre los alelos HLA II y el epítopo HTL_NS1 (RAVHADMGYWIE)
hawkdock_docking(HLA_II, 'HTL_NS1_RAVHADMGYWIE', 'HTL_NS1')
# Visualizar los IDs generados
result_HawkDock('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_HawkDock/HTL_NS1')

154531 HLAA0101_6MPPHTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:40:24 (UTC + 8 hours)
................
154532 HLAA0201_4U6XHTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:40:48 (UTC + 8 hours)
................
154533 HLAA0301_3RL1HTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:41:23 (UTC + 8 hours)
................
154534 HLAA2402_7JYVHTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:41:44 (UTC + 8 hours)
................
154535 HLAA3101HTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:42:05 (UTC + 8 hours)
................
154536 HLAB3501_4PRNHTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:42:28 (UTC + 8 hours)
................
154537 HLAB4403_3DX7HTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:42:51 (UTC + 8 hours)
................
154538 HLAB5101_1E28HTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:43:16 (UTC + 8 hours)
................
154539 HLAC0304HTL_NS1_RAVHADMGYWIE HawkDock Waiting 2023-09-25 21:43:39 (UTC + 8 hours)
................
154540 HLAC

In [7]:
# Realizar el docking entre los alelos HLA I y el epítopo HTL_NS3_1 (WDFVVTTDISEM)
hawkdock_docking(HLA_I, 'HTL_NS3_1_WDFVVTTDISEM', 'HTL_NS3_1')
# Realizar el docking entre los alelos HLA II y el epítopo HTL_NS3_1 (WDFVVTTDISEM)
hawkdock_docking(HLA_II, 'HTL_NS3_1_WDFVVTTDISEM', 'HTL_NS3_1')
# Visualizar los IDs generados
result_HawkDock('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_HawkDock/HTL_NS3_1')

147257 HLAA0101_6MPPHTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:11:43 (UTC + 8 hours)
................
147259 HLAA0201_4U6XHTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:12:04 (UTC + 8 hours)
................
147262 HLAA2402_7JYVHTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:12:45 (UTC + 8 hours)
................
147264 HLAA3101HTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:13:07 (UTC + 8 hours)
................
147264 HLAA3101HTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:13:07 (UTC + 8 hours)
................
147266 HLAB3501_4PRNHTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:13:28 (UTC + 8 hours)
................
147268 HLAB4403_3DX7HTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:13:50 (UTC + 8 hours)
................
147270 HLAB5101_1E28HTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:14:12 (UTC + 8 hours)
................
147272 HLAC0304HTL_NS3_1_WDFVVTTDISEM HawkDock Waiting 2023-07-10 03:14:33 (UTC + 8 hours)
...............

In [8]:
# Realizar el docking entre los alelos HLA I y el epítopo HTL_NS3_2 (DWDFVVTTDISE)
hawkdock_docking(HLA_I, 'HTL_NS3_2_DWDFVVTTDISE', 'HTL_NS3_2')
# Realizar el docking entre los alelos HLA II y el epítopo HTL_NS3_2 (DWDFVVTTDISE)
hawkdock_docking(HLA_II, 'HTL_NS3_2_DWDFVVTTDISE', 'HTL_NS3_2')
# Visualizar los IDs generados
result_HawkDock('/home/victor/Escritorio/Tesis/RESULTADOS/11. Docking molecular entre epítopos y los MHC de clase I y clase II/IDs_HawkDock/HTL_NS3_2')

147339 HLAA0101_6MPPHTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:08:38 (UTC + 8 hours)
................
147340 HLAA0201_4U6XHTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:09:02 (UTC + 8 hours)
................
147341 HLAA0301_3RL1HTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:09:24 (UTC + 8 hours)
................
147342 HLAA2402_7JYVHTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:09:49 (UTC + 8 hours)
................
147343 HLAA3101HTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:10:10 (UTC + 8 hours)
................
147344 HLAB3501_4PRNHTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:10:32 (UTC + 8 hours)
................
147345 HLAB4403_3DX7HTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:10:54 (UTC + 8 hours)
................
147346 HLAB5101_1E28HTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:11:15 (UTC + 8 hours)
................
147347 HLAC0304HTL_NS3_2_DWDFVVTTDISE HawkDock Waiting 2023-07-10 13:11:38 (UTC + 8 hours)
..........

Aquellos complejos con las mejores puntuaciones fueron sometidos al método de ‘**Gaussian accelerated Molecular Dynamics**’ - GaMD (*Wang et al., 2021*) en el programa ‘**NAnoscale Molecular Dynamics**’ - NAMD v2.14 (http://www.ks.uiuc.edu/Research/namd/) (*Phillips et al., 2005*) para mejorar la exploración del espacio de fase al introducir un potencial bias, al original [V(original)], V∗ = V(original) +V(GaMD), esto libre del uso de alguna coordenada de reacción (ξ) predefinida. 

Las interacciones de los residuos en el sitio de unión de los complejos eMHC fueron computadas usando la herramienta **pyContact** (*Scheurer et al., 2018*). Las interacciones fueron visualizadas con ‘**Visual Molecular Dynamics**’ - VMD v1.9.4.a (*Humphrey et al., 1996*).